In [6]:
using Random

# Определяем целевую функцию
f(x) = 5 - 24x + 17x^2 - (11/3)x^3 + (1/4)x^4

# Функция для преобразования десятичного числа в двоичное строкой фиксированной длины
function decimal_to_binary(x::Int, length::Int)
    binary_str = bitstring(x)[end-length+1:end]
    return binary_str
end

# Функция для преобразования двоичной строки в десятичное число
function binary_to_decimal(binary_str::String)
    return parse(Int, binary_str, base=2)
end

# Функция для вычисления приспособленности (fitness)
function calculate_fitness(population::Vector{String}, fitness_func)
    fitness_values = Float64[]
    for chrom in population
        x = binary_to_decimal(chrom)
        push!(fitness_values, fitness_func(x))
    end
    return fitness_values
end

# Функция для выбора родителей с использованием рулетки
function select_parents(population::Vector{String}, fitness_values::Vector{Float64})
    # Инвертируем fitness values для минимизации (так как меньшие значения лучше)
    inverted_fitness = maximum(fitness_values) .- fitness_values .+ 1e-10
    total_fitness = sum(inverted_fitness)
    probabilities = inverted_fitness ./ total_fitness
    
    parent1_idx = findfirst(cumsum(probabilities) .≥ rand())[1]
    parent2_idx = findfirst(cumsum(probabilities) .≥ rand())[1]
    
    return population[parent1_idx], population[parent2_idx]
end

# Функция для одноточечного кроссинговера
function crossover(parent1::String, parent2::String, crossover_rate::Float64)
    if rand() > crossover_rate
        return parent1, parent2
    end
    
    crossover_point = rand(1:length(parent1)-1)
    child1 = parent1[1:crossover_point] * parent2[crossover_point+1:end]
    child2 = parent2[1:crossover_point] * parent1[crossover_point+1:end]
    
    return child1, child2
end

# Функция для мутации
function mutate(child::String, mutation_rate::Float64)
    if rand() > mutation_rate
        return child
    end
    
    mutation_point = rand(1:length(child))
    mutated_child = collect(child)
    mutated_child[mutation_point] = mutated_child[mutation_point] == '0' ? '1' : '0'
    return join(mutated_child)
end

# Основная функция генетического алгоритма
function genetic_algorithm(fitness_func, population_size::Int, chromosome_length::Int, 
                          generations::Int, crossover_rate::Float64, mutation_rate::Float64,
                          x_range::Tuple{Int,Int})
    # Инициализация популяции
    min_x, max_x = x_range
    initial_population = [decimal_to_binary(rand(min_x:max_x), chromosome_length) 
                          for _ in 1:population_size]
    
    current_population = copy(initial_population)
    
    for gen in 1:generations
        # Вычисляем приспособленность
        fitness_values = calculate_fitness(current_population, fitness_func)
        
        # Выводим информацию о текущем поколении
        println("\nПоколение $gen:")
        println("Хромосома (двоичная) | Десятичное значение | Приспособленность")
        println("---------------------|---------------------|------------------")
        for (chrom, fit) in zip(current_population, fitness_values)
            x = binary_to_decimal(chrom)
            println("$chrom | $x | $fit")
        end
        
        # Создаем новую популяцию
        new_population = String[]
        
        while length(new_population) < population_size
            # Выбор родителей
            parent1, parent2 = select_parents(current_population, fitness_values)
            
            # Кроссинговер
            child1, child2 = crossover(parent1, parent2, crossover_rate)
            
            # Мутация
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)
            
            # Добавляем потомков в новую популяцию
            push!(new_population, child1)
            if length(new_population) < population_size
                push!(new_population, child2)
            end
        end
        
        current_population = new_population
    end
    
    # Находим лучшую хромосому в последнем поколении
    final_fitness = calculate_fitness(current_population, fitness_func)
    best_idx = argmin(final_fitness)
    best_chrom = current_population[best_idx]
    best_x = binary_to_decimal(best_chrom)
    best_fitness = final_fitness[best_idx]
    
    println("\nРезультат:")
    println("Лучшая хромосома: $best_chrom")
    println("Десятичное значение: $best_x")
    println("Приспособленность: $best_fitness")
    
    return best_x
end

# Параметры алгоритма
population_size = 4
chromosome_length = 3
generations = 10
crossover_rate = 0.8
mutation_rate = 0.1
x_range = (0, 7)  # Диапазон значений x

# Запуск алгоритма
best_solution = genetic_algorithm(f, population_size, chromosome_length, 
                                generations, crossover_rate, mutation_rate, x_range)


Поколение 1:
Хромосома (двоичная) | Десятичное значение | Приспособленность
---------------------|---------------------|------------------
111 | 7 | 12.583333333333485
100 | 4 | 10.333333333333343
000 | 0 | 5.0
101 | 5 | 7.916666666666686

Поколение 2:
Хромосома (двоичная) | Десятичное значение | Приспособленность
---------------------|---------------------|------------------
100 | 4 | 10.333333333333343
100 | 4 | 10.333333333333343
110 | 6 | 5.0
101 | 5 | 7.916666666666686

Поколение 3:
Хромосома (двоичная) | Десятичное значение | Приспособленность
---------------------|---------------------|------------------
001 | 1 | -5.416666666666666
101 | 5 | 7.916666666666686
110 | 6 | 5.0
101 | 5 | 7.916666666666686

Поколение 4:
Хромосома (двоичная) | Десятичное значение | Приспособленность
---------------------|---------------------|------------------
001 | 1 | -5.416666666666666
001 | 1 | -5.416666666666666
111 | 7 | 12.583333333333485
000 | 0 | 5.0

Поколение 5:
Хромосома (двоичная) | Дес

1